In [2]:
import numpy as np
import wandb

In [11]:
wandb.init(mode='disabled')

In [29]:
entity = 'cell-x'
project = 'g0-arrest-uni-cv5'

runs1 = wandb.Api().runs(f"{entity}/{project}")


In [33]:
entity = 'cell-x'
project = 'g0-arrest-resnet50-cv5'
runs2 =  wandb.Api().runs(f"{entity}/{project}")

In [40]:
def get_f1_mean_val(runs, index):
    
    run_f1_scores = {}
    run_auroc_val_scores = {}
    # Iterate over the runs
    for run in runs:
        # Retrieve the run history for the current run
        history = run.history(pandas=True)
        
        # Extract the validation F1 scores from the history
        val_f1_scores = history['f1_val'].dropna().tolist()
        auroc_val_scores = history['auroc_val'].dropna().tolist()
        
        # Store the validation F1 scores for the current run
        run_f1_scores[run.id] = val_f1_scores
        run_auroc_val_scores[run.id] = auroc_val_scores

    # # Print the validation F1 scores for each run
    # for run_id, f1_scores in run_f1_scores.items():
    #     print(f"Run ID: {run_id}")
    #     print(f"Validation F1 Scores: {f1_scores}")
    #     print()
    mean_f1_scores = []
    mean_auroc_val_scores = []
    
    index += 1
    for run_id, f1_scores in run_f1_scores.items():
        # Get the F1 score for the 7th epoch
        f1_ith_epoch = f1_scores[index]  # Assuming 0-based indexing
        
        # Append the F1 score to the list
        mean_f1_scores.append(f1_ith_epoch)
        mean_auroc_val_scores.append(run_auroc_val_scores[run_id][index])
    # Compute the mean of the F1 scores for the ith epoch
    overall_mean_f1 = np.mean(mean_f1_scores)
    overall_std_f1 = np.std(mean_f1_scores)


    overall_mean_auroc_val = np.mean(mean_auroc_val_scores)
    overall_std_auroc_val  = np.std(mean_auroc_val_scores)

    print(f"Mean F1 score for the {index}th epoch across all runs: {overall_mean_f1:.4f}")
    print(f"Std. F1 score for the {index}th epoch across all runs: {overall_std_f1:.4f}")

    print(f"Mean AUROC VAL score for the {index}th epoch across all runs: {overall_mean_auroc_val:.4f}")
    print(f"Std. AUROC VAL score for the {index}th epoch across all runs: {overall_std_auroc_val:.4f}")

In [47]:
def get_val_metrics(runs, index):
    run_f1_scores = {}
    run_auroc_val_scores = {}
    
    for run in runs:
        history = run.history(pandas=True)
        val_f1_scores = history['f1_val'].dropna().tolist()
        auroc_val_scores = history['auroc_val'].dropna().tolist()
        
        run_f1_scores[run.id] = val_f1_scores
        run_auroc_val_scores[run.id] = auroc_val_scores
    
    mean_f1_scores = [f1_scores[index] for f1_scores in run_f1_scores.values()]
    mean_auroc_val_scores = [auroc_val_scores[index] for auroc_val_scores in run_auroc_val_scores.values()]
    
    overall_mean_f1 = np.mean(mean_f1_scores)
    overall_std_f1 = np.std(mean_f1_scores)
    
    overall_mean_auroc_val = np.mean(mean_auroc_val_scores)
    overall_std_auroc_val = np.std(mean_auroc_val_scores)
    
    print(f"Mean F1 score for the {index}th epoch across all runs: {overall_mean_f1:.4f}")
    print(f"Std. F1 score for the {index}th epoch across all runs: {overall_std_f1:.4f}")
    
    print(f"Mean AUROC VAL score for the {index}th epoch across all runs: {overall_mean_auroc_val:.4f}")
    print(f"Std. AUROC VAL score for the {index}th epoch across all runs: {overall_std_auroc_val:.4f}")

### UNI

In [51]:
get_val_metrics(runs=runs1, index=8)


Mean F1 score for the 8th epoch across all runs: 0.6786
Std. F1 score for the 8th epoch across all runs: 0.0299
Mean AUROC VAL score for the 8th epoch across all runs: 0.7318
Std. AUROC VAL score for the 8th epoch across all runs: 0.0339


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [46]:
get_f1_mean_val(runs=runs1, index=5)


Mean F1 score for the 6th epoch across all runs: 0.6765
Std. F1 score for the 6th epoch across all runs: 0.0303
Mean AUROC VAL score for the 6th epoch across all runs: 0.7307
Std. AUROC VAL score for the 6th epoch across all runs: 0.0346


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


### RESNET50

In [42]:
get_f1_mean_val(runs=runs2, index=7)


Mean F1 score for the 8th epoch across all runs: 0.6691
Std. F1 score for the 8th epoch across all runs: 0.0415
Mean AUROC VAL score for the 8th epoch across all runs: 0.7426
Std. AUROC VAL score for the 8th epoch across all runs: 0.0260


In [45]:
get_f1_mean_val(runs=runs2, index=5)


Mean F1 score for the 6th epoch across all runs: 0.6752
Std. F1 score for the 6th epoch across all runs: 0.0224
Mean AUROC VAL score for the 6th epoch across all runs: 0.7388
Std. AUROC VAL score for the 6th epoch across all runs: 0.0257


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


In [52]:
wandb.finish()